<a href="https://colab.research.google.com/github/giriragav/UChicago---QCSD/blob/main/Module%203/lab6/lab6_schedulling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title
!pip install qiskit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.6 MB/s eta 0:00:00


In [2]:
# @title Imports
import numpy as np
import networkx as nx
import qiskit
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit import Qubit
from IPython.display import display

from typing import Dict, Any, List

In [3]:
# @title Data classes and help functions
# 0.2.0 Wrapping Gates to Make them hashable in a graph structure and give them a unique identifying label
import dataclasses
@dataclasses.dataclass
class GateWrapper:
    gate:Any
    qubits:List[Qubit]
    extra_params:List[Any]
    label:str

    def __hash__(self):
        return hash((type(self.gate), tuple(self.qubits), self.label))

    def __str__(self):
        args = ','.join(str(q._index) for q in self.qubits)
        return f'{self.label}{{{self.gate.name}({args})}}'

    def __repr__(self):
        return str(self)

# 0.2.1 Building a new circuit from an old circuit (This is just to see how to use gates from old circuits)
def copy_a_circuit(old_circuit, n):
    '''
    QuantumCircuit's operate on registers of qubits. If the number of qubits in old_circuit is different from the number of qubits in the new circuit
    you need to convert the qubits from old to new.

    This function explains how to copy gates from one set of qubits to another in a different circuit.

    Args:
        old_circuit: the circuit to copy
        n: number of qubits in the new circuit

    '''
    new_circuit = qiskit.QuantumCircuit(n)

    for instr in old_circuit:
        # If n = number of qubits in old_circuit this is O.K.
        # new_circuit.append(instr)

        # If n != number of qubits in old_circuit the above is NOT O.K.
        # Instead, convert the indices
        # DO NOT use both this and the above.
        new_qubit_indices = [instr.qubits[i]._index for i in range(len(instr.qubits))]
        new_circuit.append(instr.operation, new_qubit_indices, instr.clbits)

    return new_circuit

# 0.2.2 Building a Dependency Graph using NetworkX
def build_program_dependency_graph(circuit):
    '''
    Builds a program dependency graph like in the video. Feel free to modify this or build your own as you see fit.
    Here we convert the gates into their hashable versions and add a label.
    '''

    # Starting Label Index
    i = 0

    # A dictionary to store the last use of any qubit
    qubit_last_use = {}

    g = nx.DiGraph()

    # Add the start node
    g.add_node(-1)

    for instr in circuit:

        hashable_gate = GateWrapper(instr.operation, instr.qubits, instr.clbits, label=i)
        i += 1

        g.add_node(hashable_gate)

        # Add edges based on qubit_last_use; update last use
        for qubit in hashable_gate.qubits:
            if qubit in qubit_last_use:
                g.add_edge(qubit_last_use[qubit], hashable_gate)
            else:
                g.add_edge(-1, hashable_gate)

            qubit_last_use[qubit] = hashable_gate

    # Add the end node
    g.add_node(float('inf'))

    for qubit in qubit_last_use:
        g.add_edge(qubit_last_use[qubit], float('inf'))

    return g

# 0.2.3 From Dependency Graph with Hashable Gates to Qiskit Gates
def dependency_graph_to_circuit(dep_graph, n):
    '''
    Takes a dependency graph and the number of qubits n
    '''

    circuit = qiskit.QuantumCircuit(n)

    for gate in nx.topological_sort(dep_graph):

        if gate not in [-1, float('inf')]:
            circuit.append(gate.gate, gate.qubits, gate.extra_params)

    return circuit

# 0.2.4 Interaction Graphs
def interaction_graph_from_circuit(circuit):
    '''
    Builds a weighted interaction graph for a given circuit.

    Nodes are qubits
    Edges are weighted by the number of times pairs of qubits interact
    '''
    g = nx.Graph()

    for instr in circuit:
        for q in instr.qubits:
            g.add_node(q)

        for i in range(len(instr.qubits)):
            for j in range(i):
                q1 = instr.qubits[i]
                q2 = instr.qubits[j]
                if q1 != q2:
                    if (q1, q2) not in g.edges:
                        g.add_edge(q1, q2, weight=1)
                    else:
                        g.edges[q1, q2]['weight'] += 1

    return g

# 0.2.5 Checking if a gate is a certain type
basic_c = QuantumCircuit(1)
basic_c.rz(np.pi / 2, 0)
basic_c.h(0)
basic_c.x(0)

for instr in basic_c:
    if isinstance(instr.operation, qiskit.circuit.library.standard_gates.rz.RZGate):
        print("This is an RZ gate")
    if isinstance(instr.operation, qiskit.circuit.library.standard_gates.HGate):
        print("This is a H gate")
    if isinstance(instr.operation, qiskit.circuit.library.standard_gates.XGate):
        print("This is a X gate")

This is an RZ gate
This is a H gate
This is a X gate


In [4]:
# @title Visualize functions
# Helper functions to visualize
def draw_interaction_graph(ig):
    nx.draw(nx.relabel_nodes(ig, {q: f'{q._register.name}_{q._index}' for q in ig.nodes}),
            with_labels=True)

def draw_dependency_graph(dep_g):
    nx.draw(dep_g, with_labels=True)

def draw_hardware_graph(target_hardware):
    nx.draw(target_hardware, with_labels=True)

def draw_mapping(target_hardware, mapping):
    rev_mapping = {hw_q: None for hw_q in target_hardware.nodes}
    for k, v in mapping.items():
        assert v in rev_mapping, (
            'Invalid mapping: cannot map a circuit qubit to a non-existant hardware qubit '
            f'({k}->{v} but {v} does not exist)')
        assert rev_mapping[v] is None, (
            'Invalid mapping: cannot map two circuit qubits to the same qubit on hardware '
            f'({rev_mapping[v]}->{v} and {k}->{v})')
        rev_mapping[v] = k
    nx.draw(nx.relabel_nodes(target_hardware,
                             {hw_q: f'''q{getattr(rev_mapping[hw_q], '_index', None)
                                         }->{hw_q}'''
                              for hw_q in target_hardware.nodes}),
            with_labels=True)

def draw_routing(routed_circuit):
    routed_circuit.draw(fold=-1)

In [8]:
# @title Solution Cell

# For simplicity, assume all 1 qubit gates have the same execution duration
# Except, assume Z gates are free, i.e. take no time to execute
# but you should still schedule Z gates at the appropriate time between other gates

# here are some sample gate times.
sample_gate_times = {
    '1' : 10.0,
    'rz' : 0.0,  # Free
    'cx' : 100.0,
}

def schedule_circuit(routed_circuit:QuantumCircuit, gate_times:Dict[str, float]
                    ) -> Dict[GateWrapper, float]:
    '''
    This function should take an input program given as a qiskit.QuantumCircuit,
    This circuit should be routed already!

    You should return the start time for every gate in the circuit as a
    dictionary: {GateWrapper(...) : start_time}, where GateWrapper is our above
    implementation which wraps qiskit Gates.

    Hints:
        - It may be helpful to build a dependency graph weighted using these gate
        times and use nx.all_pairs_dijkstra_path_length.
        - Create the schedule keys out of qiskit gates with:
            GateWrapper(gate[0], gate[1], gate[2], label=i)  # Unique values of i
    '''
    ### YOUR SOLUTION IN THIS FUNCTION

    # Convert SWAP gates to 3 CNOT gates
    pm = qiskit.transpiler.PassManager([
        qiskit.transpiler.passes.BasisTranslator(qiskit.circuit.equivalence_library.SessionEquivalenceLibrary, ['rz', 'rx', 'h', 'u3', 'cx'])
        ])
    routed_circuit = pm.run(routed_circuit)

    # May be useful but not required
    dep_graph = build_program_dependency_graph(routed_circuit)

    # A dictionary of gate->start_time
    gate_start_times:Dict[GateWrapper, float] = {}

    # YOUR CODE HERE

    return gate_start_times

In [22]:
# @title Scheudle to Circuit
def schedule_to_circuit(schedule):
    qubits = set()
    for gate in schedule.keys():
        qubits.update(gate.qubits)
    n = len(qubits)
    print(qubits)
    circuit = QuantumCircuit(next(iter(qubits))._register)
    ordered = sorted(((time, gate)
                      for gate, time in schedule.items()),
                     key=lambda x:(x[0], x[1].gate.name!='rz'))
    for time, gate in ordered:
        new_qubits = [circuit.qubits[q._index] for q in gate.qubits]
        print(gate)
        circuit.append(gate.gate, new_qubits, [])
    return circuit


In [48]:
# @title Score Schdule
def score_schedule(routed_circuit, gate_times, schedule):
    '''
    Computes the active qubit time of the schedule.

    Returns the tuple (your score, worst-case serial score).  Lower values are better.
    '''
    #validate_schedule(routed_circuit, gate_times, schedule, check_unitary=False)
    # for gate, time in schedule.items():
        # print("-----total time calc values------")
        # print(time)
        # print(gate.gate.name)
        # print(gate_times['1'])
        # print(gate_times)
        # print("-----total time calc values------")
    total_duration = max((
        time + gate_times.get(gate.gate.name, gate_times['1'])
        for gate, time in schedule.items()),
        default=0
    )
    first_use_each_qubit = {
        q: min((time for gate, time in schedule.items()
                     if q in gate.qubits),
               default=total_duration)
        for q in routed_circuit.qubits
    }
    print("first_use_each_qubit:",first_use_each_qubit)
    qubit_usage_times = {
        q: total_duration - first_use
        for q, first_use in first_use_each_qubit.items()
    }
    print("qubit_usage_times:",qubit_usage_times)
    total_active = sum(qubit_usage_times.values())
    print("total_active:",total_active)

    num_used_qubits = sum(t > 0 for t in qubit_usage_times.values())
    print("num_used_qubits:",num_used_qubits)
    worst_serial_score = num_used_qubits * sum(
        gate_times.get(gate.gate.name, gate_times['1'])
        for gate in schedule.keys()
    )
    print(f'(Total circuit duration is {total_duration})')
    #print(f'Qubit active durations:\n{qubit_usage_times}')
    print(f'Your scheduling score is {total_active} ns, better than the worst '
          f'case score of {worst_serial_score}.')
    return total_active, worst_serial_score

In [6]:
# @title Validate Schdule
def validate_schedule(routed_circuit, gate_times, schedule, check_unitary=True):
    '''Set check_unitary=False for circuits with over 10 qubits.'''
    # Check other arguments
    assert isinstance(routed_circuit, QuantumCircuit)
    assert isinstance(gate_times, dict)
    assert isinstance(schedule, dict)
    assert (len(routed_circuit.qubits)
            == len(frozenset(q._index for q in routed_circuit.qubits))), (
        'Unsupported: circuits with multiple registers')
    # Convert schedule back to a circuit
    s_circuit = schedule_to_circuit(schedule)
    r_qubits = frozenset(q for q in routed_circuit.qubits)
    s_qubits = frozenset(q for q in routed_circuit.qubits)
    assert r_qubits == s_qubits, 'schedule has different qubits than the circuit'
    # Check allowed gates
    for gate in schedule.keys():
        if len(gate.qubits) < 2: continue
        assert len(gate.qubits) <= 2, (
            'Only one- and two-qubit gates are allowed in the schedule')
        assert isinstance(gate.gate, qiskit.circuit.library.standard_gates.x.CXGate), (
            'Two-qubit gates other than CNOT are not allwed in the schedule')
    # Check for overlapping gates
    sorted_schedule = sorted(schedule.items(), key=lambda x:x[1])
    for i, (gate, time) in enumerate(sorted_schedule):
        end_time = time + gate_times.get(gate.gate.name, gate_times['1'])
        j = 0
        for j in range(i+1, len(sorted_schedule)):
            if sorted_schedule[j][1] >= end_time: break
        nearby_gates = sorted_schedule[i+1:j]
        left = max((
            t2 + gate_times.get(g2.gate.name, gate_times['1'])
            for g2, t2 in nearby_gates
            if (t2 + gate_times.get(g2.gate.name, gate_times['1']) < end_time
                and len(set(gate.qubits).intersection(g2.qubits)) > 0)
                and g2 != gate),
            default=0
        )
        right = min((
            t2
            for g2, t2 in nearby_gates
            if ((t2 > time or (
                    gate_times.get(g2.gate.name, gate_times['1']) != 0
                    and t2 >= time))
                and len(set(gate.qubits).intersection(g2.qubits)) > 0)
                and g2 != gate),
            default=float('inf')
        )
        #print(left, right)
        assert left <= time+1e-8 and right >= end_time-1e-8, (
            'At least one gate is scheduled during the same time on the same qubits '
            f'(at time {time} - {end_time}, gate={gate})')

    # Check unitary
    if check_unitary:  # Slow for circuits with more than 10 qubits
        from qiskit.quantum_info.operators.predicates import matrix_equal
        unitary = lambda c: qiskit.quantum_info.Operator(c).data
        assert matrix_equal(unitary(s_circuit), unitary(routed_circuit)), (
            'schedule is not equivalent to the input circuit')



In [54]:
# @title Simple 1 qubit:1, Z: 2, CNOT: 2

def schedule_circuit(routed_circuit:QuantumCircuit, gate_times:Dict[str, float]
                    ) -> Dict[GateWrapper, float]:
    # Convert SWAP gates to 3 CNOT gates
    pm = qiskit.transpiler.PassManager([
        qiskit.transpiler.passes.BasisTranslator(qiskit.circuit.equivalence_library.SessionEquivalenceLibrary, ['rz', 'rx', 'h', 'u3', 'cx'])
        ])
    routed_circuit = pm.run(routed_circuit)
    dep_graph = build_program_dependency_graph(routed_circuit)

    gate_start_times:Dict[GateWrapper, float] = {}

    # YOUR CODE HERE
    for instr in routed_circuit:
      gateWrapper = GateWrapper(instr.operation, instr.qubits, [], '')
      # print("------Gatewrapper objects------")
      # print("routed circuit instruction:",instr)
      # print(gateWrapper.gate)
      # print(gateWrapper.qubits)
      gate_start_times[gateWrapper] = gate_times.get(instr.name,gate_times['1'])

    print("Gate start times:",gate_start_times)
    return gate_start_times

_routed = QuantumCircuit(4)
_routed.rz(np.pi/2, 0)
_routed.h(1)
_routed.cx(1,2)
# display(_routed.draw(fold=-1))
_gate_times = {
    '1' : 22.0,
    'rz' : 0.0,  # Free
    'cx' : 90.0,
}
_schedule = schedule_circuit(_routed, _gate_times)
display(schedule_to_circuit(_schedule).draw(fold=-1))
validate_schedule(_routed, _gate_times, _schedule)
score_schedule(_routed, _gate_times, _schedule)
print('PASS: Valid circuit schedule')

Gate start times: {{rz(0)}: 0.0, {h(1)}: 22.0, {cx(1,2)}: 90.0}
{<Qubit register=(4, "q"), index=1>, <Qubit register=(4, "q"), index=0>, <Qubit register=(4, "q"), index=2>}
{rz(0)}
{h(1)}
{cx(1,2)}


┌─────────┐     
q_0: ┤ Rz(π/2) ├─────
     └──┬───┬──┘     
q_1: ───┤ H ├─────■──
        └───┘   ┌─┴─┐
q_2: ───────────┤ X ├
                └───┘
q_3: ────────────────

{<Qubit register=(4, "q"), index=1>, <Qubit register=(4, "q"), index=0>, <Qubit register=(4, "q"), index=2>}
{rz(0)}
{h(1)}
{cx(1,2)}
first_use_each_qubit: {<Qubit register=(4, "q"), index=0>: 0.0, <Qubit register=(4, "q"), index=1>: 22.0, <Qubit register=(4, "q"), index=2>: 90.0, <Qubit register=(4, "q"), index=3>: 180.0}
qubit_usage_times: {<Qubit register=(4, "q"), index=0>: 180.0, <Qubit register=(4, "q"), index=1>: 158.0, <Qubit register=(4, "q"), index=2>: 90.0, <Qubit register=(4, "q"), index=3>: 0.0}
total_active: 428.0
num_used_qubits: 3
(Total circuit duration is 180.0)
Your scheduling score is 428.0 ns, better than the worst case score of 336.0.
PASS: Valid circuit schedule


In [ ]:
# @title GV Solution try
_routed = QuantumCircuit.from_qasm_str('OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[5];\ncx q[0],q[1];\nswap q[1],q[0];\ncx q[1],q[2];\nswap q[1],q[0];\nh q[0];\ncx q[1],q[0];\nrz(-pi/4) q[0];\nswap q[1],q[2];\ncx q[1],q[0];\nswap q[1],q[2];\nrz(pi/4) q[0];\ncx q[1],q[0];\nrz(-pi/4) q[0];\nswap q[1],q[2];\ncx q[1],q[0];\nswap q[1],q[2];\nrz(-5*pi/4) q[0];\nrx(pi/2) q[0];\nrz(pi/2) q[0];\nrz(pi/4) q[1];\ncx q[2],q[1];\nrz(pi/4) q[2];\nrz(-pi/4) q[1];\ncx q[2],q[1];\nrz(2.6375741) q[1];\nrx(pi) q[1];\nrz(2.6375741) q[1];\ncx q[2],q[1];\nswap q[1],q[0];\nswap q[2],q[1];\ncx q[2],q[3];\nswap q[2],q[1];\nswap q[1],q[0];\nh q[0];\ncx q[1],q[0];\nrz(-pi/4) q[0];\nswap q[1],q[2];\ncx q[1],q[0];\nswap q[1],q[2];\nrz(pi/4) q[0];\ncx q[1],q[0];\nrz(-pi/4) q[0];\nswap q[1],q[2];\ncx q[1],q[0];\nswap q[1],q[2];\nrz(-5*pi/4) q[0];\nrx(pi/2) q[0];\nrz(pi/2) q[0];\nrz(pi/4) q[1];\ncx q[2],q[1];\nrz(pi/4) q[2];\nrz(-pi/4) q[1];\ncx q[2],q[1];\nswap q[1],q[0];\ncx q[1],q[2];\nswap q[1],q[0];\nrz(2.6375741) q[1];\nrx(pi) q[1];\nrz(2.6375741) q[1];\ncx q[0],q[1];\n')
_gate_times = {
    '1' : 12.0,
    'rz' : 0.0,  # Free
    'cx' : 80.0,
}
_schedule = schedule_circuit(_routed, _gate_times)
validate_schedule(_routed, _gate_times, _schedule)
score_schedule(_routed, _gate_times, _schedule)
print('PASS: Valid circuit schedule')